In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

import warnings
warnings.filterwarnings('ignore')

#rb
drive_path = '/content/drive/MyDrive/Kuliah/Tugas Akhir/Final Project Shared Folder'

base_path = "/content/drive/MyDrive/Final Project Shared Folder"
base_url = "https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?CycleBeginYear=2017"
dataset_names = ['Demographics', 'Dietary', 'Examination', 'Laboratory', 'Questionnaire']

In [ ]:
data_path = "Dataset/Data Versioning/Combined_All_V1.csv"

df_raw = pd.read_csv(os.path.join(drive_path, data_path), index_col = 0)
df_raw = df_raw.set_index('SEQN')
df_raw.head()

,Demog1_RIAGENDR,Demog1_RIDAGEYR,Demog1_RIDRETH3,Demog1_DMDEDUC3,Demog1_DMDEDUC2,Demog1_DMDMARTL,Demog1_DMDHHSIZ,Demog1_DMDFMSIZ,Demog1_DMDHHSZA,Demog1_DMDHHSZB,...,Quest21_SLQ330,Quest21_SLD013,Quest22_SMQ020,Quest22_SMQ621,Quest22_SMQ890,Quest22_SMQ900,Quest23_SMD470,Quest24_WHD010,Quest24_WHD020,Quest24_WHD080M
SEQN,,,,,,,,,,,,,,,,,,,,,
93703.0,2.0,2.0,6.0,NaN,NaN,NaN,5.0,5.0,3.000000e+00,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93704.0,1.0,2.0,3.0,NaN,NaN,NaN,4.0,4.0,2.000000e+00,5.397605e-79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93705.0,2.0,66.0,4.0,NaN,2.0,3.0,1.0,1.0,5.397605e-79,5.397605e-79,...,b'07:00',8.0,1.0,NaN,2.0,2.0,NaN,63.0,165.0,NaN
93706.0,1.0,18.0,6.0,15.0,NaN,NaN,5.0,5.0,5.397605e-79,5.397605e-79,...,b'12:00',11.5,2.0,NaN,2.0,2.0,NaN,68.0,145.0,NaN
93707.0,1.0,13.0,7.0,6.0,NaN,NaN,7.0,7.0,5.397605e-79,3.000000e+00,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN


In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 9254 entries, 93703.0 to 102956.0
Columns: 150 entries, Demog1_RIAGENDR to Quest24_WHD080M
dtypes: float64(147), object(3)
memory usage: 10.7+ MB


In [ ]:
# demog_raw = df_raw[[col for col in df_raw.columns if "Demog" in col]]
# labor_raw = df_raw[[col for col in df_raw.columns if "Labor" in col]]
# exam_raw = df_raw[[col for col in df_raw.columns if "Exam" in col]]
# quest_raw = df_raw[[col for col in df_raw.columns if "Quest" in col]]
# dieta_raw = df_raw[[col for col in df_raw.columns if "Dieta" in col]]
# # demog_raw.info()

In [ ]:
def Demog1_DMDEDUC3(x):
  if x in [0,1,2,3,4,5,6,7,8,55,66]:
    return 1
  if x in [9,10,11,12]:
    return 2
  if x in [13,14]:
    return 3
  if x in [15]:
    return 4
  if x in [77]:
    return 7
  if x in [99]:
    return 9

demog = df_raw[[col for col in df_raw.columns if "Demog" in col]]

# Mapping education level for < 19 y.o to 20+ y.o education level
demog['Demog1_DMDEDUC3'] = demog['Demog1_DMDEDUC3'].apply(Demog1_DMDEDUC3)
demog['Demog1_DMDEDUC'] = demog['Demog1_DMDEDUC2'].combine_first(demog['Demog1_DMDEDUC3'])
demog = demog.drop(columns=['Demog1_DMDEDUC2', 'Demog1_DMDEDUC3'])
# The remaining null values on education level has age mean of 2 and max age of 9, set these with less than 9th grade
demog.loc[demog['Demog1_DMDEDUC'].isnull(), 'Demog1_DMDEDUC'] = 1

# Set Marital Status for < 19 y.o to never married
demog.loc[demog['Demog1_RIDAGEYR'] < 20, 'Demog1_DMDMARTL'] = 5

# Get average income from household income & family income, and set the null value with "Don't Know"
demog['Demog1_INDHHIN2'] = demog['Demog1_INDHHIN2'].combine_first(demog['Demog1_INDFMIN2'])
demog['Demog1_INDFMIN2'] = demog['Demog1_INDFMIN2'].combine_first(demog['Demog1_INDHHIN2'])
demog['Demog1_INDIN2'] = (demog['Demog1_INDHHIN2'] + demog['Demog1_INDFMIN2']) / 2
demog = demog.drop(columns=['Demog1_INDHHIN2', 'Demog1_INDFMIN2'])
demog.loc[demog['Demog1_INDIN2'].isnull(), 'Demog1_INDIN2'] = 99

demog = demog.drop(columns=['Demog1_INDFMPIR'])

demog.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 9254 entries, 93703.0 to 102956.0
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Demog1_RIAGENDR  9254 non-null   float64
 1   Demog1_RIDAGEYR  9254 non-null   float64
 2   Demog1_RIDRETH3  9254 non-null   float64
 3   Demog1_DMDMARTL  9254 non-null   float64
 4   Demog1_DMDHHSIZ  9254 non-null   float64
 5   Demog1_DMDFMSIZ  9254 non-null   float64
 6   Demog1_DMDHHSZA  9254 non-null   float64
 7   Demog1_DMDHHSZB  9254 non-null   float64
 8   Demog1_DMDHHSZE  9254 non-null   float64
 9   Demog1_DMDEDUC   9254 non-null   float64
 10  Demog1_INDIN2    9254 non-null   float64
dtypes: float64(11)
memory usage: 867.6 KB


In [ ]:
def window_mean_by_age(data, col_name, window):

  def precalculate_window_mean(data, window):
    age_map = {}
    summary = data.groupby('Demog1_RIDAGEYR').agg({col_name:['count', 'sum']})
    summary.columns = ['count', 'sum']
    summary = summary.reset_index()

    for age in summary['Demog1_RIDAGEYR'].drop_duplicates():
      age_map[age] = np.sum(summary.loc[np.logical_and(summary['Demog1_RIDAGEYR'] >= age-window, summary['Demog1_RIDAGEYR'] <= age+window), 'sum']) / np.sum(summary.loc[np.logical_and(summary['Demog1_RIDAGEYR'] >= age-window, summary['Demog1_RIDAGEYR'] <= age+window), 'count'])
    return age_map

  age_map = precalculate_window_mean(data, window)
  ret = []
  for idx, row in data.iterrows():
    ret.append(age_map[row['Demog1_RIDAGEYR']])
  return ret


In [ ]:
labor = df_raw[[col for col in df_raw.columns if "Labor" in col]]

labor

,Labor1_LBXTC,Labor1_LBDTCSI,Labor2_URXVOL1,Labor2_URDFLOW1,Labor2_URDTIME1
SEQN,,,,,
93703.0,NaN,NaN,NaN,NaN,NaN
93704.0,NaN,NaN,NaN,NaN,NaN
93705.0,157.0,4.06,248.0,1.204,206.0
93706.0,148.0,3.83,349.0,NaN,NaN
93707.0,189.0,4.89,100.0,0.526,190.0
...,...,...,...,...,...
102952.0,119.0,3.08,63.0,0.525,120.0
102953.0,182.0,4.71,63.0,0.529,119.0
102954.0,172.0,4.45,158.0,1.039,152.0


In [ ]:
# Laboratory Data

labor = df_raw[[col for col in df_raw.columns if "Labor" in col]]
col_to_fill = labor.columns

labor = labor.join(demog)

# Cleaning Cholesterol
# Participant age < 6 is not tested (refer: labor_raw.loc[demog_raw['Demog1_RIDAGEYR'] < 6].info())
# The rest have no pattern on why is the null value (Spiking null value in young age & old age), set this to mean of participant's age group.

# Cleaning Urine Flow & Time
# Participant age < 6 is not tested (refer: labor_raw.loc[demog_raw['Demog1_RIDAGEYR'] < 6].info())
# The rest have no pattern on why is the null value (Spiking null value in young age & old age), set this to mean of participant's age group. (age group to be determined)

# Cleaning Urine Volume
# Participant age < 3 is not tested (refer: labor_raw.loc[demog_raw['Demog1_RIDAGEYR'] < 3].info())
# The rest have no pattern on why is the null value (Spiking null value in young age & old age), set this to mean of participant's age group. (age group to be determined)


# for col in col_to_fill:
#   labor[col] = labor[col].fillna(labor.groupby('Demog1_RIDAGEYR')[col].transform('mean'))

for col in col_to_fill:
  labor[col] = window_mean_by_age(labor[[col, 'Demog1_RIDAGEYR']], col_name=col, window=5)

labor = labor[labor.columns.drop(list(labor.filter(regex='Demog')))]

labor

# labor.info()

,Labor1_LBXTC,Labor1_LBDTCSI,Labor2_URXVOL1,Labor2_URDFLOW1,Labor2_URDTIME1
SEQN,,,,,
93703.0,154.995595,4.008194,90.681818,0.895435,181.485507
93704.0,154.995595,4.008194,90.681818,0.895435,181.485507
93705.0,187.559570,4.850273,98.561437,1.025907,139.093882
93706.0,159.887435,4.134843,116.079936,0.783489,185.896660
93707.0,157.386364,4.069979,111.913433,0.766819,188.560329
...,...,...,...,...,...
102952.0,185.294872,4.791872,95.520446,0.913420,140.807212
102953.0,191.622971,4.955668,108.960094,0.942886,145.181452
102954.0,190.952736,4.938246,110.214286,0.949654,147.187332


In [ ]:
# Laboratory Data

labor = df_raw[[col for col in df_raw.columns if "Labor" in col]]
col_to_fill = labor.columns

labor = labor.join(demog)

# Cleaning Cholesterol
# Participant age < 6 is not tested (refer: labor_raw.loc[demog_raw['Demog1_RIDAGEYR'] < 6].info())
# The rest have no pattern on why is the null value (Spiking null value in young age & old age), set this to mean of participant's age group.

# Cleaning Urine Flow & Time
# Participant age < 6 is not tested (refer: labor_raw.loc[demog_raw['Demog1_RIDAGEYR'] < 6].info())
# The rest have no pattern on why is the null value (Spiking null value in young age & old age), set this to mean of participant's age group. (age group to be determined)

# Cleaning Urine Volume
# Participant age < 3 is not tested (refer: labor_raw.loc[demog_raw['Demog1_RIDAGEYR'] < 3].info())
# The rest have no pattern on why is the null value (Spiking null value in young age & old age), set this to mean of participant's age group. (age group to be determined)


for col in col_to_fill:
  labor[col] = labor[col].fillna(labor.groupby('Demog1_RIDAGEYR')[col].transform('mean'))

# for col in col_to_fill:
#   labor[col] = window_mean_by_age(labor[[col, 'Demog1_RIDAGEYR']], col_name=col, window=5)

labor = labor[labor.columns.drop(list(labor.filter(regex='Demog')))]

labor

# labor.info()

,Labor1_LBXTC,Labor1_LBDTCSI,Labor2_URXVOL1,Labor2_URDFLOW1,Labor2_URDTIME1
SEQN,,,,,
93703.0,NaN,NaN,NaN,NaN,NaN
93704.0,NaN,NaN,NaN,NaN,NaN
93705.0,157.0,4.06,248.0,1.204000,206.000000
93706.0,148.0,3.83,349.0,0.827314,176.447619
93707.0,189.0,4.89,100.0,0.526000,190.000000
...,...,...,...,...,...
102952.0,119.0,3.08,63.0,0.525000,120.000000
102953.0,182.0,4.71,63.0,0.529000,119.000000
102954.0,172.0,4.45,158.0,1.039000,152.000000


In [ ]:
labor

,Labor1_LBXTC,Labor1_LBDTCSI,Labor2_URXVOL1,Labor2_URDFLOW1,Labor2_URDTIME1
SEQN,,,,,
93703.0,NaN,NaN,NaN,NaN,NaN
93704.0,NaN,NaN,NaN,NaN,NaN
93705.0,157.0,4.06,248.0,1.204000,206.000000
93706.0,148.0,3.83,349.0,0.827314,176.447619
93707.0,189.0,4.89,100.0,0.526000,190.000000
...,...,...,...,...,...
102952.0,119.0,3.08,63.0,0.525000,120.000000
102953.0,182.0,4.71,63.0,0.529000,119.000000
102954.0,172.0,4.45,158.0,1.039000,152.000000


In [ ]:
# Examination Data

exam = df_raw[[col for col in df_raw.columns if "Exam" in col]]
col_to_fill = exam.columns

exam = exam.join(demog)

# BPAARM --> might need to discard, desc: "Arm selected"

# Blood Pressure Data (BPXPLS, BPXPTY, BPXSY1, BPXDI1, BPXSY2, BPXDI2, BPXSY3, BPXDI3)
# Users with age < 8 have null data (ref: exam_raw.loc[demog_raw['Demog1_RIDAGEYR'] < 8].info())

# Body measure data
# Might need to use average for each age group or average of data (Don't use too long data window) of the users age.

for col in col_to_fill:
  exam[col] = exam[col].fillna(exam.groupby('Demog1_RIDAGEYR')[col].transform('mean'))

exam = exam[exam.columns.drop(list(exam.filter(regex='Demog')))]

exam.info()


<class 'pandas.core.frame.DataFrame'>
Float64Index: 9254 entries, 93703.0 to 102956.0
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Exami1_BPAARM    7545 non-null   float64
 1   Exami1_BPXPLS    7545 non-null   float64
 2   Exami1_BPXPULS   9254 non-null   float64
 3   Exami1_BPXPTY    7545 non-null   float64
 4   Exami1_BPXSY1    7545 non-null   float64
 5   Exami1_BPXDI1    7545 non-null   float64
 6   Exami1_BPXSY2    7545 non-null   float64
 7   Exami1_BPXDI2    7545 non-null   float64
 8   Exami1_BPXSY3    7545 non-null   float64
 9   Exami1_BPXDI3    7545 non-null   float64
 10  Exami2_BMXWT     9254 non-null   float64
 11  Exami2_BMXHT     8663 non-null   float64
 12  Exami2_BMXBMI    8663 non-null   float64
 13  Exami2_BMXLEG    7545 non-null   float64
 14  Exami2_BMXARML   9254 non-null   float64
 15  Exami2_BMXARMC   9254 non-null   float64
 16  Exami2_BMXWAIST  8663 non-null   float64
 17  Ex

In [ ]:
quest = df_raw[[col for col in df_raw.columns if "Quest" in col]]

quest = quest.join(demog)

group_to_drop = [
    'Quest5', # Consumer Behavior Phone Follow-up Module - Adult Questions. Question is too wild, might not be applicable in app, example question: Buy food from fast food/pizza places, Notice calories at fast food/pizza place, Eat at all-you-can-eat buffet , Notice calories at recreational places, etc.
    'Quest6', # Dermatology Question, not sure if relevant, might need to bring back
    'Quest13', # Hospital Utilization & Access to Care Questions, only have 1 col to use (How long since last healthcare visit) which have 86% null
    'Quest21', # Sleep disorders questions. Might need to bring back, not sure why dropped
    'Quest24', # Weight History Questions, can be derived from weight data from examination.
]

cols_to_drop = ['Quest1_ALQ121', # Past 12 mo how often have alcohol drink, might need to bring back
                'Quest2_BPD035', # Age told had hypertension, data is continuous and only have value if the Quest2_BPQ020 is yes. [Inconsistent Context when Fill NaN] How to clean without giving the wrong context to the model? e.g. if null value fill with 0, then it could mean they had hypertension at age 0?
                'Quest2_BPQ040A', # Taking prescription for hypertension, might be duplicate context as Quest2_BPQ020
                'Quest2_BPQ060', # Ever had blood cholesterol checked, might need to bring back
                'Quest2_BPQ070', # When blood cholesterol last checked, Inconsistent Context when Fill NaN
                'Quest2_BPQ100D', # Now taking prescribed medicine, lot of nulls, might be duplicate context as Quest2_BPQ020
                'Quest7_DIQ175B', # Overweight, lot of nulls, can be derived from weight on examination data
                'Quest7_DIQ175C', # Age, lot of nulls, can be get from weight on examination data
                'Quest7_DIQ175G', # Lack of physical activity, ~96% null.
                'Quest7_DIQ175H', # High blood pressure, ~96% null, might be duplicated context as Quest2_BPQ020
                'Quest7_DIQ175I', # High blood sugar, 99% null
                'Quest7_DIQ175J', # High cholesterol, 98% null
                'Quest7_DIQ175N', # Blurred vision, 98% null
                'Quest7_DIQ175T', # Frequent urination, 98% null
                'Quest7_DIQ175V', # Craving for sweet/eating a lot of sugar, 99% null
                'Quest14_INQ150', # Income from other sources, might need to bring back
                'Quest18_OCQ260', # Description of job/work situation, values are too wild, might not be effective & applicable in the application (example values: A federal government employee, A state government employee, Working without pay in family business or farm, etc.)
                'Quest19_PAD615', # Minutes vigorous-intensity work, might be duplicate context as PAQ610 (Number of days vigorous work)
                'Quest19_PAQ640', # Number of days walk or bicycle, might be duplicate context as PAQ635 (Walk or bicycle). might need to bring back
                'Quest19_PAD645', # Minutes walk/bicycle for transportation, duplicate context as PAQ640
                'Quest19_PAQ655', # Days vigorous recreational activities, might not applicable in app, might need to bring back
                'Quest19_PAD660', # Minutes vigorous recreational activities, duplicate context as PAQ655
                'Quest22_SMQ621' # Cigarettes smoked in entire life, 87% null, same context as other cigarettes question (or not?)
              ]
cont_fill_mean = [
    {'name': 'Quest4_CBD111', 'replace': [777777, 999999]},
    {'name': 'Quest4_CBD121', 'replace': [777777, 999999]},
    {'name': 'Quest18_OCQ180', 'replace': [77777, 99999]},
    {'name': 'Quest19_PAQ610', 'replace': [77, 99]}

]
bool_value_map = {
    'Quest1_ALQ111': '9',
    'Quest2_BPQ020': '9',
    'Quest3_CDQ008': '9',
    'Quest3_CDQ010': '9',
    'Quest7_DIQ010': '9',
    'Quest9_DLQ050': '9',
    'Quest10_ECQ020': '9',
    'Quest11_HIQ011': '9',
    'Quest12_HEQ010': '9',
    'Quest12_HEQ030': '9',
    'Quest14_INQ020': '9',
    'Quest14_INQ012': '9',
    'Quest14_IND235': '99',
    'Quest15_KIQ022': '9',
    'Quest15_KIQ026': '9',
    'Quest17_DPQ020': '9',
    'Quest17_DPQ030': '9',
    'Quest17_DPQ040': '9',
    'Quest18_OCQ210': '9',
    'Quest19_PAQ635': '9',
    'Quest20_PFQ020': '9',
    'Quest20_PFQ061B': '9',
    'Quest20_PFQ061C': '9',
    'Quest20_PFQ061H': '9',
    'Quest22_SMQ020': '9',
    'Quest22_SMQ890': '9',
    'Quest22_SMQ900': '9',
}
special_treatment_cols = [
    'Quest3_CDQ009A',
    'Quest3_CDQ009B',
    'Quest3_CDQ009C',
    'Quest3_CDQ009D',
    'Quest3_CDQ009E',
    'Quest3_CDQ009F',
    'Quest3_CDQ009G',
    'Quest3_CDQ009H',
    'Quest23_SMD470'
]

for group in group_to_drop:
  quest = quest[quest.columns.drop(list(quest.filter(regex=group)))]

quest = quest.drop(columns = [col for col in quest.columns if col in cols_to_drop])

for col in cont_fill_mean:
  for val in col['replace']:
    quest[col['name']] = quest[col['name']].replace(val, None)
  quest[col['name']] = quest[col['name']].fillna(quest.groupby('Demog1_RIDAGEYR')[col['name']].transform('mean'))

for col, val in bool_value_map.items():
  quest[col] = quest[col].replace(int(re.sub('9', '7', val)), None)
  quest[col] = quest[col].fillna(9)

quest['Quest3_CDQ009'] = quest['Quest3_CDQ009A'].notnull() | quest['Quest3_CDQ009B'].notnull() | quest['Quest3_CDQ009C'].notnull() | quest['Quest3_CDQ009D'].notnull() | quest['Quest3_CDQ009E'].notnull() | quest['Quest3_CDQ009F'].notnull() | quest['Quest3_CDQ009G'].notnull() | quest['Quest3_CDQ009H'].notnull()
quest['Quest3_CDQ009'] = quest['Quest3_CDQ009'].replace({True: 1, False: 0}).astype(float)

def family_smoker(x):
  if x in (1,2,3):
    return 1
  return 0

quest['Quest23_SMD470'] = quest['Quest23_SMD470'].apply(family_smoker)

quest = quest.drop(columns = [col for col in quest.columns if col in special_treatment_cols])

quest = quest[quest.columns.drop(list(quest.filter(regex='Demog')))]

quest.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 9254 entries, 93703.0 to 102956.0
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Quest1_ALQ111    9254 non-null   float64
 1   Quest2_BPQ020    9254 non-null   float64
 2   Quest3_CDQ008    9254 non-null   float64
 3   Quest3_CDQ010    9254 non-null   float64
 4   Quest4_CBD111    9254 non-null   float64
 5   Quest4_CBD121    9254 non-null   float64
 6   Quest7_DIQ010    9254 non-null   float64
 7   Quest9_DLQ050    9254 non-null   float64
 8   Quest10_ECQ020   9254 non-null   float64
 9   Quest11_HIQ011   9254 non-null   float64
 10  Quest12_HEQ010   9254 non-null   float64
 11  Quest12_HEQ030   9254 non-null   float64
 12  Quest14_INQ020   9254 non-null   float64
 13  Quest14_INQ012   9254 non-null   float64
 14  Quest14_IND235   9254 non-null   float64
 15  Quest15_KIQ022   9254 non-null   float64
 16  Quest15_KIQ026   9254 non-null   float64
 17  Qu

In [ ]:
dieta = df_raw[[col for col in df_raw.columns if "Dieta" in col]]
col_to_fill = dieta.columns

dieta = dieta.join(demog)

for col in col_to_fill:
  dieta[col] = dieta[col].fillna(dieta.groupby('Demog1_RIDAGEYR')[col].transform('mean'))

dieta = dieta[dieta.columns.drop(list(dieta.filter(regex='Demog')))]

dieta.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 9254 entries, 93703.0 to 102956.0
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Dieta1_DRDINT    9254 non-null   float64
 1   Dieta1_DR1DAY    9254 non-null   float64
 2   Dieta1_DR1FS     9254 non-null   float64
 3   Dieta1_DR1IFDCD  9254 non-null   float64
 4   Dieta1_DR1IGRMS  9254 non-null   float64
 5   Dieta1_DR1IKCAL  9254 non-null   float64
 6   Dieta1_DR1IPROT  9254 non-null   float64
 7   Dieta1_DR1ICARB  9254 non-null   float64
 8   Dieta1_DR1ISUGR  9254 non-null   float64
 9   Dieta1_DR1IFIBE  9254 non-null   float64
 10  Dieta1_DR1ITFAT  9254 non-null   float64
 11  Dieta1_DR1ISFAT  9254 non-null   float64
 12  Dieta1_DR1IMFAT  9254 non-null   float64
 13  Dieta1_DR1IPFAT  9254 non-null   float64
 14  Dieta1_DR1ICHOL  9254 non-null   float64
 15  Dieta1_DR1ICALC  9254 non-null   float64
 16  Dieta2_DRDINT    9254 non-null   float64
 17  Di

In [ ]:
# Null is because some question are not meant for person below certain ages, but for all respondent with age above 16 have no null values (Except Dietary)

final = demog.copy()
final = final.join(labor)
final = final.join(exam)
final = final.join(quest)
final = final.join(dieta)

# final = final[final['Demog1_RIDAGEYR'] >= 18]

final = final.reset_index()

final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9254 entries, 0 to 9253
Data columns (total 96 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SEQN             9254 non-null   float64
 1   Demog1_RIAGENDR  9254 non-null   float64
 2   Demog1_RIDAGEYR  9254 non-null   float64
 3   Demog1_RIDRETH3  9254 non-null   float64
 4   Demog1_DMDMARTL  9254 non-null   float64
 5   Demog1_DMDHHSIZ  9254 non-null   float64
 6   Demog1_DMDFMSIZ  9254 non-null   float64
 7   Demog1_DMDHHSZA  9254 non-null   float64
 8   Demog1_DMDHHSZB  9254 non-null   float64
 9   Demog1_DMDHHSZE  9254 non-null   float64
 10  Demog1_DMDEDUC   9254 non-null   float64
 11  Demog1_INDIN2    9254 non-null   float64
 12  Labor1_LBXTC     7878 non-null   float64
 13  Labor1_LBDTCSI   7878 non-null   float64
 14  Labor2_URXVOL1   8421 non-null   float64
 15  Labor2_URDFLOW1  7878 non-null   float64
 16  Labor2_URDTIME1  7878 non-null   float64
 17  Exami1_BPAARM 

In [ ]:
target_path = "Dataset/Data Versioning/Combined_All_V3.csv"

final.to_csv(os.path.join(drive_path, target_path))